<a href="https://colab.research.google.com/github/FilipeSCampos/Data_restore_scrapping/blob/main/Trabalho_Web_Scrapping_certo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
HTMLFileToBeOpened = open("Etherium_Data_24_20.html", "r")
mainpage = HTMLFileToBeOpened.read()
soup = BeautifulSoup(mainpage, 'lxml')


#Duas formas de fazer webscraping uma baixando o html da pagina (Muito mais facil) e outra do jeito comum em request

In [ ]:
print(soup.find('tbody'))


In [ ]:
#ocultado pq é grande pra caramba

#Olhando o cod html na pagina da pra ver que os valores estao dentro da <tbody>

In [ ]:
# primeira coluna
print(soup.find('tbody').find_all('td')[0].prettify())
# segunda coluna
print(soup.find('tbody').find_all('td')[1].prettify())
for col in range (7):
    print(soup.find('tbody').find_all('td')[col].prettify())

#Tendo Certeza que ta tudo certo

In [ ]:
# vendo as 7 primeiras colunas em 3 dias
for row in range(3):
    for col in range(7):
        print(soup.find('tbody').find_all('td')[row*7+col].text.strip())
    print()

#Transformando em DataFrame

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# Definir o intervalo de datas desejado (três anos atrás até a data atual)
end_date = datetime.now()
start_date = end_date - timedelta(days=365 * 3)

# Inicializa listas para armazenar os dados
dates = []
prices = []

# Itera sobre as linhas correspondentes ao intervalo de datas desejado
rows = soup.find('tbody').find_all('tr')
for row in rows:
    # Extrai a data e o preço das três primeiras colunas
    cols = row.find_all('td')
    date_str = cols[0].text.strip()
    price = cols[1].text.strip()

    # Converte a string de data para objeto datetime
    date = datetime.strptime(date_str, '%b %d, %Y')

    # Verifica se a data está dentro do intervalo desejado
    if start_date <= date <= end_date:
        # Adiciona os dados às listas
        dates.append(date)
        prices.append(price)

# Cria um DataFrame com os dados coletados
df = pd.DataFrame({'Date': dates, 'Price': prices})



In [ ]:
df

In [ ]:
df.info()

#Uma Breve Série Temporal para analise do preço da bitcoin em relação ao tempo


In [ ]:
# Remover o símbolo de dólar ($) e quaisquer outros caracteres não numéricos da coluna de preços
df['Price'] = df['Price'].str.replace(r'[^0-9.]+', '', regex=True)

# Converter a coluna de preços para o tipo float
df['Price'] = df['Price'].astype(float)

# Agora podemos plotar o gráfico de linha temporal
import matplotlib.pyplot as plt

# Converter a coluna de data para o tipo datetime
df['Date'] = pd.to_datetime(df['Date'])

# Ordenar o DataFrame pela data (do mais antigo para o mais recente)
df = df.sort_values(by='Date')

# Plotar o gráfico de linha temporal
plt.figure(figsize=(12, 6))
plt.plot(df['Date'], df['Price'], marker='o', linestyle='-')
plt.title('Tendência do Preço ao Longo do Tempo')
plt.xlabel('Data')
plt.ylabel('Preço')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


#Tentativas de conseguir exito por meio de requests

In [ ]:
import requests
url = "https://coinmarketcap.com/currencies/ethereum/historical-data/"
response = requests.get(url)
if response.status_code == 200:
    print("A solicitação foi bem-sucedida")
else:
    print("Ocorreu um erro ao fazer a solicitação.")


#Procurando no html da pagina onde ficava o preco, e vasculhando onde cada div table e classe ficava, uma dentro da outra até chegar na tbody

In [ ]:
table = soup.find("table", {"class": "cmc-table"})
if table:
    rows = table.find_all("tr")
    # Continue o código para extrair os dados das linhas da tabela
else:
    print("Não foi possível encontrar a tabela.")


In [ ]:
# Encontrar a div que contém a tabela
history_div = soup.find("div", class_="history")
if history_div:
    # Dentro da div "history", encontrar a div que contém a tabela
    table_div = history_div.find("div", class_="sc-4033ac70-2 gUDHzc")
    if table_div:
        # Dentro da div da tabela, encontrar a tabela propriamente dita
        table = table_div.find("table")
        if table:
            # Dentro da tabela, encontrar todas as linhas
            rows = table.find_all("tr")
            # Agora você pode continuar a extrair os dados das células das linhas
        else:
            print("Não foi possível encontrar a tabela.")
    else:
        print("Não foi possível encontrar a div da tabela.")
else:
    print("Não foi possível encontrar a div 'history'.")


In [ ]:
print(history_div)

In [ ]:
#grande pra caramba

In [ ]:
# Encontrar a div "history" dentro do corpo da página
history_div = soup.find("div", class_="history")
if history_div:
    # Dentro da div "history", encontrar a div que contém a tabela
    table_div = history_div.find("div", class_="sc-4033ac70-2")
    if table_div:
        # Dentro da div da tabela, encontrar a tabela propriamente dita
        table = table_div.find("table")
        if table:
            # Dentro da tabela, encontrar todas as linhas
            rows = table.find_all("tr")
            # Agora você pode continuar a extrair os dados das células das linhas
        else:
            print("Não foi possível encontrar a tabela.")
    else:
        print("Não foi possível encontrar a div da tabela.")
else:
    print("Não foi possível encontrar a div 'history' dentro do corpo da página.")


In [ ]:
print(rows)

In [ ]:
#grande pra caramba

#Encontrando os dados finalmente

In [ ]:
# Dentro da tabela, encontrar todas as linhas
rows = table.find_all("tr")

# Lista para armazenar os dados da tabela
dados_tabela = []

# Iterar sobre as linhas, ignorando o cabeçalho
for row in rows[1:]:
    # Encontrar todas as células (colunas) nesta linha
    cells = row.find_all("td")

    # Extrair o conteúdo de cada célula e adicioná-lo à lista de dados
    if len(cells) >= 2:
        produto = cells[0].text.strip()
        preco = cells[1].text.strip()
        dados_tabela.append((produto, preco))

# Exibir os dados extraídos
print("Dados da tabela:")
for produto, preco in dados_tabela:
    print(f"Produto: {produto}, Preço: {preco}")


In [ ]:
import pandas as pd

# Criar um DataFrame a partir dos dados extraídos
df = pd.DataFrame(dados_tabela, columns=["Data", "Preco"])

# Exibir o DataFrame
print("DataFrame:")
print(df)


In [ ]:
df.info()

In [ ]:
import pandas as pd

# Supondo que você tenha uma coluna de datas chamada "Data" no DataFrame df
# Convertendo a coluna "Data" para o tipo datetime
df["Data"] = pd.to_datetime(df["Data"])
# Remover o símbolo de dólar ($) e quaisquer outros caracteres não numéricos da coluna de preços
df['Preco'] = df['Preco'].str.replace(r'[^0-9.]+', '', regex=True)

# Converter a coluna de preços para o tipo float
df['Preco'] = df['Preco'].astype(float)

# Exibir o DataFrame atualizado
print("DataFrame com índice de data:")
print(df)


In [ ]:
# Agora podemos plotar o gráfico de linha temporal
import matplotlib.pyplot as plt

# Ordenar o DataFrame pela data (do mais antigo para o mais recente)
df = df.sort_values(by='Data')

# Plotar o gráfico de linha temporal
plt.figure(figsize=(12, 6))
plt.plot(df['Data'], df['Preco'], marker='o', linestyle='-')
plt.title('Tendência do Preço ao Longo do Tempo')
plt.xlabel('Data')
plt.ylabel('Preço')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
